In [1]:
from common.data_pipeline.MMCBNU_6000.dataset import DatasetLoader as mmcbnu
from common.data_pipeline.FV_USM.dataset import DatasetLoader as fvusm
from common.util.data_pipeline.dataset_chainer import DatasetChainer
from common.util.enums import EnvironmentType

datasets = DatasetChainer(datasets=[mmcbnu(included_portion=0.001), fvusm(included_portion=0)])
train, test, validation = datasets.get_dataset(
    EnvironmentType.TENSORFLOW,
)

2023-08-22 14:10:41,390 - root - INFO - Compiled train/test/validation split for MMCBNU6000
2023-08-22 14:10:41,466 - root - INFO - Compiled train/test/validation split for FV_USM
2023-08-22 14:10:41,468 - root - INFO - Concatenating train set
2023-08-22 14:10:41,469 - root - INFO - Concatenating test set
2023-08-22 14:10:41,471 - root - INFO - Concatenating validation set


In [2]:
from common.util.enums import DatasetSplitType


datasets.get_files(DatasetSplitType.TRAIN)

[DatasetObject(path='./datasets/MMCBNU_6000/ROIs/065/L_Middle/10.bmp', name='065/L_Middle/10.bmp', label=None, mask_path=None, metadata={'finger': 'middle', 'hand': 'left', 'is_augmented': False}),
 DatasetObject(path='./datasets/MMCBNU_6000/ROIs/052/L_Ring/02.bmp', name='052/L_Ring/02.bmp', label=None, mask_path=None, metadata={'finger': 'ring', 'hand': 'left', 'is_augmented': False}),
 DatasetObject(path='./datasets/MMCBNU_6000/ROIs/037/R_Fore/07.bmp', name='037/R_Fore/07.bmp', label=None, mask_path=None, metadata={'finger': 'fore', 'hand': 'right', 'is_augmented': False}),
 DatasetObject(path='./datasets/MMCBNU_6000/ROIs/051/R_Fore/04.bmp', name='051/R_Fore/04.bmp', label=None, mask_path=None, metadata={'finger': 'fore', 'hand': 'right', 'is_augmented': False})]